In [1]:
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import xmltodict
import json
import numpy as np
import gc

In [2]:
class_dict=dict()
class_dict['normal']=0
class_dict['assault']=1
class_dict['fight']=2
class_dict['burglary']=3
class_dict['vandalism']=4
class_dict['swoon']=5
class_dict['wander']=6
class_dict['trespass']=7
class_dict['dump']=8
class_dict['robbery']=9
class_dict['datefight']=10
class_dict['kidnap']=11
class_dict['drunken']=12

In [3]:
def prepro(xml_path, video_file, num_seqence=16, fps_setting=5, action=0):
    xml_path = xml_path
    video_file = video_file
    num_seq = num_seqence
    fps_set = fps_setting
    
    #xml 파싱
    f = open(xml_path, 'r')
    read = f.read()
    dict2_type = xmltodict.parse(read)

    label=dict()
    key_pos=[0,0]

    st = dict2_type['annotation']['event']['starttime'].split(':')
    du = dict2_type['annotation']['event']['duration'].split(':')

    stint = int(((float(st[0])*3600) + (float(st[1])*60) + float(st[2])) * int(dict2_type['annotation']['header']['fps']))
    etint = stint + int(((float(du[0])*3600) + (float(du[1])*60) + float(du[2])) * int(dict2_type['annotation']['header']['fps']))


    #크롭 및 리사이즈            
    width = int(dict2_type['annotation']['size']['width'])
    height = int(dict2_type['annotation']['size']['height'])
    depth = int(dict2_type['annotation']['size']['depth'])

    if width>height:
        low=height
    else:
        low=width

    xs = key_pos[0]-int(low/2)
    xe = key_pos[0]+int(low/2)
    ys = key_pos[1]-int(low/2)
    ye = key_pos[1]+int(low/2)

    if xs < 0:
        xe = xe - xs
        xs = 0
    if ys < 0:
        ye = ye - ys
        ys=0
    if xe > width:
        xs = xs - (xe - width)
        xe = width
    if ye > height:
        ys = ys - (ye - height)
        ys = height


    start_end=[]
    for obj in label:
        #print(obj)
        for act in label[obj]:
            #print(act)
            for se in label[obj][act]:
                #print(se)
                start_end.append([act, se[0], se[1]])
                
    fps_dev = int(int(dict2_type['annotation']['header']['fps'])/fps_set)
    cap = cv2.VideoCapture(video_file)
    img_arr = []
    img_frame = []
    i=0

    if cap.isOpened():
        while True:
            i+=1
            ret, img = cap.read()
            if ret:
                if i%fps_dev==0:
                    img_frame.append(i)
                    cropped_img = img[ys: ye, xs: xe]
                    resized_img = cv2.resize(cropped_img, dsize=(172, 172), interpolation=cv2.INTER_AREA)
                    img_arr.append(resized_img)
                if i%100==0:
                    print(i)
                    gc.collect()
            else:
                break
    else:
        print('cannot open the file')

    cla = class_dict[dict2_type['annotation']['event']['eventname']]
    
    
    train = []
    label = []

    for i in range(num_seq, len(img_arr)):
        train.append(img_arr[i-num_seq:i])
        if img_frame[i] > stint and img_frame[i] < etint:
            label.append(cla)
        else:
            label.append(0)

    train2 = np.array(train, dtype=np.uint8)
    label2 = np.array(label, dtype=np.uint8)

    np.save('./sampledata/sample_train_'+ str(action) +'_uint8.npy', train2)
    np.save('./sampledata/sample_label_'+ str(action) +'_uint8.npy', label2)

In [ ]:
prepro('./이상행동 CCTV 영상/03.절도(burglary)/insidedoor_02/457-3_burglary02_place04_day/457-3_cam01_burglary02_place04_day_spring.xml',
      './이상행동 CCTV 영상/03.절도(burglary)/insidedoor_02/457-3_burglary02_place04_day/457-3_cam01_burglary02_place04_day_spring.mp4',
      16,5,3)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
